In [1]:
cd ..

/Users/dreamgonfly/Documents/pozalabs/infinityWord


In [2]:
from datasets.seq2seq_datasets import Seq2SeqIndexedDataset

In [3]:
from models.transformer_encoder import TransformerEncoder
from models.transformer_decoder import TransformerDecoder

In [4]:
from trainers.seq2seq_trainer import Seq2SeqTrainer, seq2seq_collate_fn

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

In [6]:
from utils import get_logger

In [7]:
from dictionaries import BaseDictionary

In [8]:
from models.seq2seq_model import Seq2SeqModel

In [9]:
from trainers.sorted_2d_batch_sampler import Sorted2DBatchSampler, Sorted2DBatchSamplerOnTheFly

In [10]:
dictionary = BaseDictionary.load('base_dictionary')
dictionary.vocabulary_size

48034

In [11]:
device = torch.device('cpu')

In [12]:

embeddings = nn.Embedding(num_embeddings=dictionary.vocabulary_size, embedding_dim=512)
encoder = TransformerEncoder(num_layers=3, d_model=512, heads=8, d_ff=512, dropout=0.2, embeddings=embeddings)
decoder = TransformerDecoder(num_layers=3, d_model=512, heads=8, d_ff=512, dropout=0.2, embeddings=embeddings)
model = Seq2SeqModel(encoder, decoder)

logger = get_logger(log_name='transformer')
train_dataset = Seq2SeqIndexedDataset('train')
val_dataset = Seq2SeqIndexedDataset('val')

trainer = Seq2SeqTrainer(model=model, 
                         train_dataloader=DataLoader(train_dataset, 
                                                     batch_sampler=Sorted2DBatchSamplerOnTheFly(train_dataset, batch_size=32, drop_last=False, max_length=100), 
                                                     collate_fn=seq2seq_collate_fn), 
                         val_dataloader=DataLoader(val_dataset, 
                                                   batch_sampler=Sorted2DBatchSamplerOnTheFly(val_dataset, batch_size=100, drop_last=False, max_length=100),
                                                   collate_fn=seq2seq_collate_fn), 
                         loss_function=CrossEntropyLoss(reduce=False),
                         optimizer=Adam(model.parameters()),
                         device=device,
                         logger=logger)

In [13]:
trainer.run(epochs=10)

  0%|          | 0/817 [00:00<?, ?it/s]

sources torch.Size([32, 19])


sources torch.Size([100, 7])
sources torch.Size([100, 6])
sources torch.Size([100, 4])
sources torch.Size([100, 11])
sources torch.Size([100, 9])
sources torch.Size([100, 6])
sources torch.Size([100, 11])
sources torch.Size([100, 7])
sources torch.Size([100, 4])
sources torch.Size([100, 7])
sources torch.Size([100, 10])
sources torch.Size([100, 5])
sources torch.Size([100, 4])
sources torch.Size([100, 11])
sources torch.Size([100, 6])
sources torch.Size([100, 9])
sources torch.Size([100, 4])
sources torch.Size([100, 4])
sources torch.Size([100, 5])


KeyboardInterrupt: 

In [13]:
ls parameters/Seq2SeqModel/

Seq2SeqModel_0_2018-08-04 19:57:19.028198.pth
Seq2SeqModel_0_2018-08-04 20:34:28.266519.pth
Seq2SeqModel_0_2018-08-05 14:10:28.401894.pth
Seq2SeqModel_0_2018-08-05 14:23:30.982203.pth


In [14]:
from generators.base_generators import CandidatesGenerator

In [15]:
from generators.beam import Beam

In [20]:
generator = CandidatesGenerator(model=model,
                    preprocess=lambda source: [dictionary[word] for word in sum([sentence.split() for sentence in source], [])],
                    postprocess=lambda h: ' '.join([dictionary.idx2word[token_id] for token_id in h[:-1]]), # exclude EndSent token
                    checkpoint_filepath='parameters/Seq2SeqModel/Seq2SeqModel_0_2018-08-05 14:23:30.982203.pth',
                    max_length=30,
                    beam_size=8
                   )

In [21]:
model.encoder.num_layers

3

In [22]:
a = torch.tensor([[1,2,3,4,56,7,8,9,9]])

In [23]:
generator.generate_candidates(['버려진 섬마다 꽃이 피었다', '그 꽃들은 정말 아름다운 꽃이었다'], n_candidates=5)

source_tensor torch.Size([1, 9])
encoder_state: (num_layers, batch_size=1, hidden_size) torch.Size([9, 1, 512])
memory_bank: (batch_size, seq_len, hidden_size) torch.Size([1, 9, 512])
decoder_state src torch.Size([1, 9])
previous_input previous_input None
previous_input previous_layer_inputs  None
torch.Size([8])


['확실해.', '매해', '컸기', '아이에게', '⎡으악!등청!']